### 1. Tạo pertured data và lưu giống data cũ (cal_metric.py)
    1. Tạo ra câu pertured va convert json và ghép label (pertured_folder/masked_data_json)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. đưa json (masked data 0.1) vào model lấy logit, prediction -> cần tạo class để lấy cho cả origin data và masked data
### 3. convert label gold theo từ thành label gold theo token
### 4. Tính inf 

## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [ ]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 85